# APERTURE PHOTOMETRY OF CENTRE - Abhiram

## All required Imports

### Let's start with R-Band

In [3]:
import numpy as np
import astropy
import photutils
import ccdproc
from ccdproc import CCDData, combiner
from astropy import units as u
import astropy.io.fits as fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from photutils import centroid_com, centroid_1dg, centroid_2dg
from photutils import CircularAperture
from photutils import aperture_photometry
from photutils import Background2D
from photutils import MedianBackground
from photutils import DAOStarFinder
from photutils import detect_sources, deblend_sources, source_properties
from scipy.ndimage import shift
import gc                               

from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia


C:\Users\abhir\AppData\Local\Temp/ipykernel_39108/2729152076.py:12: DeprecationWarning: `photutils.centroid_com` is a deprecated alias for `photutils.centroids.centroid_com`. Instead, please use `from photutils.centroids import centroid_com` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\abhir\AppData\Local\Temp/ipykernel_39108/2729152076.py:12: DeprecationWarning: `photutils.centroid_1dg` is a deprecated alias for `photutils.centroids.centroid_1dg`. Instead, please use `from photutils.centroids import centroid_1dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\abhir\AppData\Local\Temp/ipykernel_39108/2729152076.py:12: DeprecationWarning: `photutils.centroid_2dg` is a deprecated alias for `photutils.centroids.centroid_2dg`. Instead, please use `from photutils.centroids import centroid_2dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg


In [4]:
scim=CCDData.read("NGC_1851_R_median.fits", unit="adu") #gets the image CCD data
#finds mean median and standard deviation of the counts in the image
med=np.median(scim.data)
scim.data=scim.data-med
mean, median, std = astropy.stats.sigma_clipped_stats(scim.data, sigma=3.0, maxiters=5)
print('Image stats (mean, median and standard deviation):', mean,median,std)


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'. [astropy.wcs.wcs]


Image stats (mean, median and standard deviation): -0.15485499799251556 -0.6407623291015625 19.123193740844727


In [5]:
#outfile="segimage.fits" # Set the output file name
segimage=detect_sources(scim.data, 2.00*std, 9, connectivity=4, mask=None)
source_table=source_properties(scim.data, segimage, error=None, mask=None, background=None, filter_kernel=None, wcs=None, labels=None)
print('Source table length:\n', len(source_table))
print('First entry centroid y and x values :') # The dreaded y-x
print(source_table[0].centroid[0].value, source_table[0].centroid[1].value)

Source table length:
 310
First entry centroid y and x values :
3.1075333974370665 65.26611632758353


        Use `~photutils.segmentation.SourceCatalog` instead. [warnings]
        Use `~photutils.segmentation.SourceCatalog` instead. [photutils.segmentation.properties]
        Use `~photutils.segmentation.SourceCatalog` instead. [photutils.segmentation.properties]


In [6]:
positions=[]
for obj in source_table:
    positions.append((obj.centroid[1].value, obj.centroid[0].value))
print(positions[0:10]) # Print example values from the positions list.
#get the aperture photometry
apertures = CircularAperture(positions,r=20.0)
phot_table = aperture_photometry(segimage, apertures)
print(phot_table) #print the table of values

[(65.26611632758353, 3.1075333974370665), (1619.463528413904, 1.1170243108636084), (1629.7999668366256, 4.153575838244375), (1645.070495185167, 1.4559479636635846), (1504.949147176421, 4.193140130542071), (1635.969876293759, 2.641410884990731), (1612.730480455442, 3.5726296006671103), (1625.047625810064, 5.918017566425519), (1441.6641289348534, 5.441037257350498), (1615.0340528018392, 9.43404872588139)]
 id      xcenter            ycenter          aperture_sum   
           pix                pix                           
--- ------------------ ------------------ ------------------
  1  65.26611632758353 3.1075333974370665               85.0
  2  1619.463528413904 1.1170243108636084  763.2667080172358
  3 1629.7999668366256  4.153575838244375 1193.1691800563124
  4  1645.070495185167 1.4559479636635846 1101.6882006923734
  5  1504.949147176421  4.193140130542071               60.0
  6  1635.969876293759  2.641410884990731 1133.3121008855924
  7  1612.730480455442 3.5726296006671103  6

### Filter out values to find the centre point in the gobular cluster.
### When comparing with ds9, we found centre point to be at (593,485)
### We used a circular aperture with a high radius from the centre, as our cluster was fairly circular in shape.

In [7]:
centre_pos = []
#Filtering our values that are definitely not in the centre of the cluster
for i in positions:
    if (i[0] > 450 and i[0] < 730) and (i[1] > 370 and i[1] < 620):
        centre_pos.append(i)
print(centre_pos,len(centre_pos))

#Find aperture for centre
#The radius was incremented until the increase the increase in aperture_sum was getting lower.
#We settled with a radius of 130 pixels from the centre
apertures = CircularAperture(centre_pos[0],r=130.0)
phot_table = aperture_photometry(scim.data, apertures)
print(phot_table) #print the table of values
phot_table_R_centre = phot_table

[(593.0326968039201, 485.28742686000066), (619.4959812325171, 373.0618190509543), (529.1069442304283, 374.10235069140737), (677.0911832271095, 381.30545588412406), (689.3454099197778, 382.8671055203079), (622.4702870228356, 380.63893507744757), (708.3853288630966, 380.31884240651357), (597.6599363965024, 385.3660349193457), (591.588631186349, 385.1440886517325), (608.3464941657069, 390.916462817393), (669.2843514325363, 395.41534729254556), (474.3525534581793, 404.1421222609255), (701.826717274485, 411.89229357710786), (481.83827639495394, 412.27492241361807), (485.09940880039903, 416.92317512968623), (725.2213446795703, 420.3583601763231), (485.0052574992566, 436.75054339063524), (706.908278444544, 434.81924854003137), (467.6654130589967, 442.08550920729357), (712.4446294197672, 443.15582964374534), (720.7661382232023, 459.2836341046418), (487.05920113787465, 477.46485783649285), (471.22206130171173, 490.2719385310334), (721.6120646252303, 497.790507752179), (717.4507285728191, 509.06

### Let's do the same thing for the shifted V band image

In [8]:
scim=CCDData.read("NGC_1851_V_median_latest.fits", unit="adu") #gets the image CCD data
#finds mean median and standard deviation of the counts in the image

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'. [astropy.wcs.wcs]


In [9]:
positions=[]
for obj in source_table:
    positions.append((obj.centroid[1].value, obj.centroid[0].value))
print(positions[0:10]) # Print example values from the positions list.

centre_pos = []
#Filtering our values that are definitely not in the centre of the cluster
for i in positions:
    if (i[0] > 450 and i[0] < 730) and (i[1] > 370 and i[1] < 620):
        centre_pos.append(i)
print(centre_pos,len(centre_pos))

apertures = CircularAperture(centre_pos[0], r=130.0)
phot_table = aperture_photometry(scim.data, apertures)
print(phot_table)
phot_table_V_centre = phot_table

[(65.26611632758353, 3.1075333974370665), (1619.463528413904, 1.1170243108636084), (1629.7999668366256, 4.153575838244375), (1645.070495185167, 1.4559479636635846), (1504.949147176421, 4.193140130542071), (1635.969876293759, 2.641410884990731), (1612.730480455442, 3.5726296006671103), (1625.047625810064, 5.918017566425519), (1441.6641289348534, 5.441037257350498), (1615.0340528018392, 9.43404872588139)]
[(593.0326968039201, 485.28742686000066), (619.4959812325171, 373.0618190509543), (529.1069442304283, 374.10235069140737), (677.0911832271095, 381.30545588412406), (689.3454099197778, 382.8671055203079), (622.4702870228356, 380.63893507744757), (708.3853288630966, 380.31884240651357), (597.6599363965024, 385.3660349193457), (591.588631186349, 385.1440886517325), (608.3464941657069, 390.916462817393), (669.2843514325363, 395.41534729254556), (474.3525534581793, 404.1421222609255), (701.826717274485, 411.89229357710786), (481.83827639495394, 412.27492241361807), (485.09940880039903, 416.9

### Now let's find the magnitudes and colour of this part

In [10]:
V_centre_value = phot_table_V_centre[0]['aperture_sum']
R_centre_value = phot_table_R_centre[0]['aperture_sum']
print(V_centre_value,R_centre_value)

#For R-band
f1 = 72470 # from catalogue
f2 = R_centre_value
m1 = 12.834967068627794 # from astrometry
R_mag = -2.5 * np.log10(f2/f1) + m1

#for V-band
f1 = 14693 # from catalogue
f2 = V_centre_value
m1 = 13.575232871866096 # from astrometry
V_mag = -2.5 * np.log10(f2/f1) + m1

print(R_mag,V_mag,V_mag-R_mag)


14262583.594952025 14767077.147990452
7.062126362471496 6.107513561125105 -0.954612801346391
